# Dijkstra's Algorithm
In this exercise, you'll implement Dijkstra's algorithm. First, let's build the graph.
## Graph Representation
In order to run Dijkstra's Algorithm, we'll need to add distance to each edge. We'll use the `GraphEdge` class below to represent each edge between a node.

In [2]:
class GraphEdge(object):
    def __init__(self, node, distance):
        self.node = node
        self.distance = distance

The new graph representation should look like this:

In [3]:
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.edges = []

    def add_child(self, node, distance):
        self.edges.append(GraphEdge(node, distance))

    def remove_child(self, del_node):
        if del_node in self.edges:
            self.edges.remove(del_node)
        
    # newly added, in order to use heapq
    def __gt__(self, node):
        return self.value > node.value

class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list

    def add_edge(self, node1, node2, distance):
        if node1 in self.nodes and node2 in self.nodes:
            node1.add_child(node2, distance)
            node2.add_child(node1, distance)

    def remove_edge(self, node1, node2):
        if node1 in self.nodes and node2 in self.nodes:
            node1.remove_child(node2)
            node2.remove_child(node1)

Now let's create the graph.

In [4]:
node_u = GraphNode('U')
node_d = GraphNode('D')
node_a = GraphNode('A')
node_c = GraphNode('C')
node_i = GraphNode('I')
node_t = GraphNode('T')
node_y = GraphNode('Y')

graph = Graph([node_u, node_d, node_a, node_c, node_i, node_t, node_y])
graph.add_edge(node_u, node_a, 4)
graph.add_edge(node_u, node_c, 6)
graph.add_edge(node_u, node_d, 3)
graph.add_edge(node_d, node_u, 3)
graph.add_edge(node_d, node_c, 4)
graph.add_edge(node_a, node_u, 4)
graph.add_edge(node_a, node_i, 7)
graph.add_edge(node_c, node_d, 4)
graph.add_edge(node_c, node_u, 6)
graph.add_edge(node_c, node_i, 4)
graph.add_edge(node_c, node_t, 5)
graph.add_edge(node_i, node_a, 7)
graph.add_edge(node_i, node_c, 4)
graph.add_edge(node_i, node_y, 4)
graph.add_edge(node_t, node_c, 5)
graph.add_edge(node_t, node_y, 5)
graph.add_edge(node_y, node_i, 4)
graph.add_edge(node_y, node_t, 5)

## Implementation
Using what you've learned, implement Dijkstra's Algorithm to find the shortest distance from the "U" node to the "Y" node. 

In [12]:
import math
from heapq import heappop, heappush, heapify

'''
NOTE: Please see my final implementation in the following cells. The one in this cell has some
shortcommings as described in the comments.
UPDATE: The best latest and greatest solution is in final cell.
'''

class HeapNode:
    def __init__(self, node, distance):
        self.node = node
        self.distance = distance
        
    def __gt__(self, node):
        return self.distance > node.distance

def dijkstra(start_node, end_node):
    
    graph_node_to_heap_node = dict()
    h = []
    
    graph_node_to_heap_node[start_node] = HeapNode(start_node, 0)
    heappush(h, graph_node_to_heap_node[start_node])
    
    while len(h) > 0:
        current_heap_node = heappop(h)
        if current_heap_node.node == end_node: # when end_node is popped, return distance
            return current_heap_node.distance
        
        for edge in current_heap_node.node.edges:
            if edge.node not in graph_node_to_heap_node:
                graph_node_to_heap_node[edge.node] = HeapNode(edge.node, current_heap_node.distance + edge.distance)
                heappush(h, graph_node_to_heap_node[edge.node])
            else:
                heap_node = graph_node_to_heap_node[edge.node]
                if heap_node.distance > current_heap_node.distance + edge.distance:
                    heap_node.distance = current_heap_node.distance + edge.distance
                    heapify(h) # Instead of doing this heapify to update the heap order, I can just
# push a new heap node with updated distance again to the heap. Because, heapify() represents
# build-heap operation and could be O(N), where inserting a new node to the heap would be O(logN). 
# But now, if I were to insert another node, there will be two nodes in the heap
# corresponding to same graph node, and we don't want to explore the node two times. We only want 
# to explore it (as in look at its children), when it is popped the first time that corresponds to 
# smaller distace from start_node. So, we can maintain a set of explored nodes, and if a popped node
# is explored already, we continue. 
                    
# Also interestingly, now that I don't want to update some attribute of heap-node and then, heapify().
# I can just use a tuple inseated of my HeapNode class, that would save some memmory. I had decided to
# create the class earlier, since tuple is immutable in Python. 

#UPDTE: even solution is next cell is not very clean. See the following cell for cleanest solution.

print('Shortest Distance from {} to {} is {}'.format(node_u.value, node_y.value, dijkstra(node_u, node_y)))

Shortest Distance from U to Y is 14


One shortcomming of the previous implementation as described in comments inside the function have led me to re-implementing the stuff in following manner.

In [13]:
# This solution works but is not very clean, see next cell.
def dijkstra(start_node, end_node):
    
    node_to_smallest_distance = dict() # dicionary from node to smallest-distance-from-start_node
    h = [] # heap
    explored_nodes = set() # set of explored nodes
    
    node_to_smallest_distance[start_node] = 0
    heappush(h, (0, start_node)) # in order to do this, I have added a custom comparator to
    # class GraphNode, as if distances of two heap nodes match, heapq would try to push in 
    # lexicographic order and for that it would demand that the objects of class GraphNode be comparable.
    
    while len(h) > 0:
        current_distance_from_start, current_node = heappop(h)
        if current_node in explored_nodes:
            continue
        
        explored_nodes.add(current_node) 
        
        for edge in current_node.edges:
            new_distance = current_distance_from_start + edge.distance
            
            if edge.node not in node_to_smallest_distance:
                node_to_smallest_distance[edge.node] = new_distance
                heappush(h, (new_distance, edge.node))
            else:
                prev_distance = node_to_smallest_distance[edge.node]
                if prev_distance > new_distance:
                    node_to_smallest_distance[edge.node] = new_distance
                    heappush(h, (new_distance, edge.node))

    return node_to_smallest_distance[end_node] # can also return, as soon as end node is popped
# from the heap to save ourselves from exploring further nodes

print('Shortest Distance from {} to {} is {}'.format(node_u.value, node_y.value, dijkstra(node_u, node_y)))

Shortest Distance from U to Y is 14


In [14]:
# This is a clean solution, along the same lines implementation of A-star. Basically, don’t need to maintain a 
# dictionary of smallest distance, can simply push to heap with new distance, if it’s smaller, it’ll be popped first. 
# Also, added the check that push to heap only if not explored.

def dijkstra(start_node, end_node):
    
    h = [] # heap
    explored_nodes = set() # set of explored nodes
    
    heappush(h, (0, start_node)) # in order to do this, I have added a custom comparator to
    # class GraphNode, as if distances of two heap nodes match, heapq would try to push in 
    # lexicographic order and for that it would demand that the objects of class GraphNode be comparable.
    
    while len(h) > 0:
        current_distance_from_start, current_node = heappop(h)
        if current_node in explored_nodes:
            continue
        
        if end_node == current_node:
            return current_distance_from_start
        
        explored_nodes.add(current_node) 
        
        for edge in current_node.edges:
            if edge.node in explored_nodes:
                continue
                
            new_distance = current_distance_from_start + edge.distance
            heappush(h, (new_distance, edge.node))
            
print('Shortest Distance from {} to {} is {}'.format(node_u.value, node_y.value, dijkstra(node_u, node_y)))

Shortest Distance from U to Y is 14


<span class="graffiti-highlight graffiti-id_6vmf0hp-id_cjtybve"><i></i><button>Show Solution</button></span>